# (09) Ablation -- neural

**Motivation**: Lesion bottom-up and top-down pathways, test how much performance drops. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-MTMST/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-MTMST/figs')
tmp_dir = os.path.join(git_dir, 'jb-MTMST/tmp')

# Code path
sys.path.insert(0, '/home/hadi/Documents/MTMST/code')
from vae.train_vae import TrainerVAE, ConfigTrainVAE
from vae.vae2d import VAE, ConfigVAE
from figures.fighelper import *
from analysis.glm import *


# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from rich.jupyter import print
%matplotlib inline
set_style()

## Fit perm scores

Code adapted from ```09_neural-analysis-KABA``` notebook.

In [2]:
path = results_dir()
path = pathlib.Path(path)

trainer_paths = set()
pat = '**/*fixate1*/**/Trainer'
for p in path.rglob(pat):
    if p.is_dir():
        trainer_paths.add(str(p))
trainer_paths = sorted(trainer_paths)
len(trainer_paths)

66

In [3]:
selected_fits = [
    s for s in trainer_paths if
    int(s.split('(')[1].split(')')[0].split('_')[1]) == 8
]
selected_fits

['/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.15_(2023_08_08,03:01)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.15_lesion-dec-2_(2023_08_08,00:38)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.15_lesion-dec-4_(2023_08_08,01:26)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.15_lesion-dec-8_(2023_08_08,02:13)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.15_lesion-enc-2_(2023_08_07,22:05)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.15_lesion-enc-4_(2023_08_07,23:00)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.15_lesion-enc-8_(2023_08_07,23:48)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.5_(2023_08_08,15:39)/Trainer',
 '/home/hadi/Documents/MTMST/results/Ridge/fixate1_nf-420_beta-0.5_lesion-dec-2_(2023_08_08,12:30)/Trainer',
 '/home/hadi/Documents/MTMST/results/R

In [4]:
# %%time

pbar = tqdm(selected_fits)
for fit_path in pbar:
    fit_name = fit_path.split('/')[-2]
    f = pjoin(tmp_dir, 'neural_analysis', f"{fit_name}.npy")
    if os.path.isfile(f):
        continue
    pbar.set_description(fit_name)

    df, df_all, ro_all, args, tr = summarize_neural_fits(
        fit_name=fit_name, device='cuda')

    # extract neural results
    neural_results = collections.defaultdict(dict)
    for key, ro in ro_all.items():
        if hasattr(ro, 'kws_xtract'):
            ro.kws_push = {
                k: v for k, v in
                ro.kws_push.items()
                if 'xtract' not in k
            }
        else:
            ro.kws_xtract = ro.kws_push['kws_xtract']
            ro.kws_push = {
                k: v for k, v in
                ro.kws_push.items()
                if 'xtract' not in k
            }
        for cell, m in ro.mod.items():
            data = ro.validate(cell)
            pred = m.predict(data['x'])
            if ro.has_repeats:
                pred_tst = m.predict(data['x_tst'])
            else:
                pred_tst = None

            # perm importance
            perm = sk_inspect.permutation_importance(
                estimator=m,
                X=data['x_tst' if ro.has_repeats else 'x'],
                y=data['y_tst' if ro.has_repeats else 'y'],
                random_state=0,
                n_repeats=10,
            )
            perm_mu = np.maximum(0, perm['importances_mean'])
            perm_sd = perm['importances_std']

            # Probs
            p_z_given_neuron = perm_mu / perm_mu.sum()
            a = 1 - entropy_normalized(p_z_given_neuron, axis=0)
            
            k = f"{key}_{cell}"
            neural_results['data'][k] = data
            neural_results['pred'][k] = pred
            neural_results['pred_tst'][k] = pred_tst
            neural_results['perm_mu'][k] = perm_mu
            neural_results['perm_sd'][k] = perm_sd
            neural_results['alignment_score'][k] = a

    # save
    save_obj(
        obj=neural_results,
        file_name=fit_name,
        save_dir=pjoin(tmp_dir, 'neural_analysis'),
        verbose=True,
        mode='npy',
    )
    torch.cuda.empty_cache()

fixate1_nf-420_beta-0.5_(2023_08_08,15:39):   0%|        | 0/14 [00:00<?, ?it/s]/home/hadi/Documents/MTMST/code/analysis/helper.py:130: RuntimeWarning: invalid value encountered in true_divide
  return signal_power / response_power


[PROGRESS] 'fixate1_nf-420_beta-0.5_(2023_08_08,15:39).npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/neural_analysis

fixate1_nf-420_beta-0.5_lesion-dec-2_(2023_08_08,12:30):  57%|▌| 8/14 [43:10<32:/home/hadi/Documents/MTMST/code/analysis/helper.py:130: RuntimeWarning: invalid value encountered in true_divide
  return signal_power / response_power


[PROGRESS] 'fixate1_nf-420_beta-0.5_lesion-dec-2_(2023_08_08,12:30).npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/neural_analysis

fixate1_nf-420_beta-0.5_lesion-dec-4_(2023_08_08,13:32):  64%|▋| 9/14 [1:26:52<5/home/hadi/Documents/MTMST/code/analysis/helper.py:130: RuntimeWarning: invalid value encountered in true_divide
  return signal_power / response_power


[PROGRESS] 'fixate1_nf-420_beta-0.5_lesion-dec-4_(2023_08_08,13:32).npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/neural_analysis

fixate1_nf-420_beta-0.5_lesion-dec-8_(2023_08_08,14:35):  71%|▋| 10/14 [2:08:06</home/hadi/Documents/MTMST/code/analysis/helper.py:130: RuntimeWarning: invalid value encountered in true_divide
  return signal_power / response_power


[PROGRESS] 'fixate1_nf-420_beta-0.5_lesion-dec-8_(2023_08_08,14:35).npy' saved at
/home/hadi/Dropbox/git/jb-MTMST/tmp/neural_analysis

fixate1_nf-420_beta-0.5_lesion-dec-8_(2023_08_08,14:35): 100%|█| 14/14 [2:49:15<
